In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

PROJECT_DIR = "/content/drive/My Drive/philosophy_argument_mining"
os.makedirs(PROJECT_DIR, exist_ok=True)


In [ ]:
!pip install -q transformers accelerate bitsandbytes auto-gptq

In [ ]:
!git clone https://github.com/miheerp33/philosophy_project_1.git
%cd philosophy_project_1

# Load preprocessed paragraphs
with open("data/clean_paragraphs.txt", "r", encoding="utf-8") as f:
    paragraphs = [p.strip() for p in f.read().split("\n\n") if len(p.strip()) > 30]


fatal: destination path 'philosophy_project_1' already exists and is not an empty directory.
/content/philosophy_project_1


In [ ]:
!pip install optimum

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_pr

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# ─────────────────────────────────────────────────────────────
# STEP 4: Prompt + Generate Argument Structure
# ─────────────────────────────────────────────────────────────
def extract_argument(paragraph):
    prompt = f"""Extract any arguments from this paragraph. For each argument, list:

- Premises
- Conclusion

Paragraph:
\"\"\"{paragraph}\"\"\"

Output:"""
    result = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)[0]["generated_text"]
    return result.split("Output:")[-1].strip()


In [ ]:
# ─────────────────────────────────────────────────────────────
# STEP 5: Run on Sample and Save to Drive
# ─────────────────────────────────────────────────────────────
import json

outputs = []
for i, p in enumerate(paragraphs[:10]):  # process first 10 paragraphs for now
    print(f"\n--- Paragraph {i+1} ---")
    result = extract_argument(p)
    print(result)
    outputs.append({
        "paragraph": p,
        "extracted_argument": result
    })

# Save to JSON in Google Drive
with open(os.path.join(PROJECT_DIR, "arguments_output.json"), "w") as f:
    json.dump(outputs, f, indent=2)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Paragraph 1 ---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[
  {
    "premises": ["A few words about Dostoevsky himself"],
    "conclusion": ["may help the English reader to understand his work"]
  }
]

# This paragraph does not contain an argument. It is just an introductory statement. An argument requires at least one premise and a conclusion. In this case, there is only one statement, which does not logically imply a conclusion.

--- Paragraph 2 ---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

"""Argument 1:
Premises:
- Dostoevsky experienced intense suffering
- He accepted every suffering with resignation and regarded it as a blessing
- He constantly recurs to the subject of suffering in his writings
- He describes the awful agony of the condemned man
- He insists on the cruelty of inflicting such torture
Conclusion:
- Dostoevsky's religious temper led him to accept suffering as a blessing, but he was deeply affected by it and wrote extensively about it, emphasizing its cruelty."""

"""Argument 2:
Premises:
- Dostoevsky spent four years in penal servitude in Siberia
- He began writing "The Dead House" during this time
Conclusion:
- Dostoevsky's experiences in Siberian penal servitude influenced his writing, particularly the work "The Dead House"."""

--- Paragraph 8 ---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Argument 1:
- The man suffered from epilepsy after his arrest.
- The man experienced periods of great strain which increased the frequency of his epileptic fits.
- The man returned to Russia in 1859 and started a journal, "Vremya," which was forbidden by the censorship due to a misunderstanding.
- The man lost his first wife and brother in 1864.
- The man was in poverty and took upon himself the payment of his brother's debts.
- The man started another journal, "The Epoch," which was also prohibited.
- The man was weighed down by debt and had to write at heart-breaking speed without correcting his work.
- Conclusion: The man's epileptic fits occurred more frequently during periods of great strain and poverty.

Argument 2:
- The man's journal, "Vremya," was forbidden by the censorship through a misunderstanding.
- The man's journal, "The Epoch," was also prohibited.
- Conclusion: The man's journals were frequently prohibited.

--- Paragraph 9 ---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Argument 1:
- The month was June 1880.
- He made a famous speech.
- He was received with extraordinary demonstrations of love and honour.

Conclusion:
- He made a famous speech and was received with extraordinary demonstrations of love and honour.

--- Paragraph 10 ---
Argument 1:
- Dostoevsky died a few months ago.
- Dostoevsky had a vast multitude of mourners at his funeral.
- Dostoevsky is still the most widely read writer in Russia.

Conclusion: Dostoevsky is still the most widely read writer in Russia.
